# Breakthrough Model Training - With Shop Assortment Features

This notebook reproduces the breakthrough performance (CV R² ≈ 0.2947) and incorporates new shop assortment features to achieve further improvement (target CV R² ≈ 0.3112).

**Key Steps:**
1. Load and preprocess historical data.
2. Aggregate data to unique product-employee combinations.
3. **NEW:** Perform EDA and engineer shop-level assortment proxy features (non-leaky).
4. Prepare final feature set (original + new shop features) and target variables (including log transformation).
5. Apply stratified cross-validation.
6. Train XGBoost model with optimal parameters.
7. Evaluate performance and feature importance.

## 1. Setup and Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import pickle
import joblib

# Add src to path for imports if any utilities were there
# sys.path.append('../') 

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully")

## 2. Data Loading and Initial Preprocessing

In [ ]:
print("[DATA] Loading historical selection data...")
data_path = "../src/data/historical/present.selection.historic.csv"
try:
    raw_data = pd.read_csv(data_path, encoding='utf-8', dtype='str')
    print(f"Loaded {len(raw_data)} selection events from {data_path}")
    print(f"Columns: {list(raw_data.columns)}")
    # display(raw_data.head()) # Keep display for interactive runs
except FileNotFoundError:
    print(f"ERROR: {data_path} not found.")
    raw_data = pd.DataFrame() 

if not raw_data.empty:
    print("\n[CLEAN] Cleaning data...")
    cleaned_data = raw_data.copy()
    for col in cleaned_data.columns:
        cleaned_data[col] = cleaned_data[col].astype(str).str.strip('"').str.strip()
    cleaned_data = cleaned_data.fillna("NONE")
    categorical_cols_to_lower = ['employee_gender', 'product_target_gender', 
                       'product_utility_type', 'product_durability', 'product_type']
    for col in categorical_cols_to_lower:
        if col in cleaned_data.columns:
            cleaned_data[col] = cleaned_data[col].str.lower()
    print(f"Data cleaning complete: {len(cleaned_data)} records")
    # display(cleaned_data.head())
else:
    print("Raw data is empty, skipping cleaning.")
    cleaned_data = pd.DataFrame()

## 3. Data Aggregation (Unique Product-Employee Combinations)

In [ ]:
agg_data = pd.DataFrame() # Initialize
grouping_cols = []
if not cleaned_data.empty:
    grouping_cols = [
        'employee_shop', 'employee_branch', 'employee_gender',
        'product_main_category', 'product_sub_category', 'product_brand',
        'product_color', 'product_durability', 'product_target_gender',
        'product_utility_type', 'product_type'
    ]
    print(f"\n[AGGREGATE] Grouping by {len(grouping_cols)} features: {grouping_cols}")
    agg_data = cleaned_data.groupby(grouping_cols).size().reset_index(name='selection_count')
    if not agg_data.empty:
        compression_ratio = len(cleaned_data) / len(agg_data) if len(agg_data) > 0 else 0
        print(f"Aggregation complete:")
        print(f"  {len(cleaned_data)} events → {len(agg_data)} unique combinations")
        print(f"  Compression ratio: {compression_ratio:.1f}x")
        # display(agg_data.head())
    else:
        print("agg_data is empty after grouping.")
else:
    print("Cleaned data is empty, skipping aggregation.")

## 4. Shop Assortment Feature Engineering (EDA and New Feature Creation)

### 4.1 EDA for Shop-Level Patterns

In [ ]:
shop_summary = pd.DataFrame()
shop_top_main_cats = pd.DataFrame()
shop_top_brands = pd.DataFrame()
agg_data_with_shop_features = pd.DataFrame() # Initialize

if not agg_data.empty:
    print("\nPerforming EDA for Shop-Level Patterns...")
    shop_summary = agg_data.groupby('employee_shop').agg(
        total_shop_selections=('selection_count', 'sum'),
        unique_product_combinations_in_shop=('product_main_category', 'count'),
        distinct_main_categories_in_shop=('product_main_category', 'nunique'),
        distinct_sub_categories_in_shop=('product_sub_category', 'nunique'),
        distinct_brands_in_shop=('product_brand', 'nunique'),
        distinct_utility_types_in_shop=('product_utility_type', 'nunique')
    ).reset_index()
    print("\nShop Summary Statistics (EDA):")
    # display(shop_summary.head())
    # display(shop_summary.describe())

    shop_main_cat_counts = agg_data.groupby(['employee_shop', 'product_main_category'])['selection_count'].sum().reset_index()
    idx = shop_main_cat_counts.groupby(['employee_shop'])['selection_count'].transform(max) == shop_main_cat_counts['selection_count']
    shop_top_main_cats = shop_main_cat_counts[idx].drop_duplicates(subset=['employee_shop'], keep='first')
    shop_top_main_cats = shop_top_main_cats[['employee_shop', 'product_main_category']].rename(
        columns={'product_main_category': 'shop_most_frequent_main_category_selected'}
    )
    # print("\nShop Most Frequent Main Category (Selected):")
    # display(shop_top_main_cats.head())

    shop_brand_counts = agg_data.groupby(['employee_shop', 'product_brand'])['selection_count'].sum().reset_index()
    idx_brand = shop_brand_counts.groupby(['employee_shop'])['selection_count'].transform(max) == shop_brand_counts['selection_count']
    shop_top_brands = shop_brand_counts[idx_brand].drop_duplicates(subset=['employee_shop'], keep='first')
    shop_top_brands = shop_top_brands[['employee_shop', 'product_brand']].rename(
        columns={'product_brand': 'shop_most_frequent_brand_selected'}
    )
    # print("\nShop Most Frequent Brand (Selected):")
    # display(shop_top_brands.head())
    print("EDA for shop patterns complete.")
else:
    print("Skipping EDA as agg_data is empty.")

### 4.2 Engineer Shop-Level Aggregate Features (Non-Leaky)

In [ ]:
shop_features_df = pd.DataFrame() # Initialize

if not shop_summary.empty:
    print("\nEngineering Shop-Level Aggregate Features...")
    shop_features_df = shop_summary.copy()
    if not shop_top_main_cats.empty:
        shop_features_df = pd.merge(shop_features_df, shop_top_main_cats, on='employee_shop', how='left')
    if not shop_top_brands.empty:
        shop_features_df = pd.merge(shop_features_df, shop_top_brands, on='employee_shop', how='left')
    
    # Rename columns for clarity (some were already good from shop_summary)
    shop_features_df = shop_features_df.rename(columns={
        'distinct_main_categories_in_shop': 'shop_main_category_diversity_selected',
        'distinct_brands_in_shop': 'shop_brand_diversity_selected',
        'distinct_utility_types_in_shop': 'shop_utility_type_diversity_selected',
        'distinct_sub_categories_in_shop': 'shop_sub_category_diversity_selected' # Added this for completeness
    })
    # Drop features that might be too close to target if used directly or redundant after other calculations
    # 'total_shop_selections' and 'shop_avg_selections_per_product_combination' were found to be problematic.
    # We will rely on diversity and frequency features instead.
    cols_to_potentially_drop_from_shop_features = ['total_shop_selections']
    # Example: also drop 'shop_avg_selections_per_product_combination' if it was created in shop_summary and needs removal
    # if 'shop_avg_selections_per_product_combination' in shop_features_df.columns: 
    #    cols_to_potentially_drop_from_shop_features.append('shop_avg_selections_per_product_combination')
    shop_features_df = shop_features_df.drop(columns=[col for col in cols_to_potentially_drop_from_shop_features if col in shop_features_df.columns], errors='ignore')

    print("Shop-level features created and refined:")
    # display(shop_features_df.head())
    
    if not agg_data.empty:
        agg_data_with_shop_features = pd.merge(agg_data, shop_features_df, on='employee_shop', how='left')
        print("\nAgg_data with shop-level features merged.")
        # display(agg_data_with_shop_features.head())
    else:
        print("agg_data is empty, cannot merge shop features.")
else:
    print("Skipping Shop-Level Aggregate Features as shop_summary is empty.")
    agg_data_with_shop_features = agg_data.copy() # Proceed with original agg_data if no shop features

### 4.3 Engineer Product-Relative-to-Shop Features (Non-Leaky)

In [ ]:
final_features_df = pd.DataFrame() # Initialize

if not agg_data_with_shop_features.empty:
    print("\nEngineering Product-Relative-to-Shop Features (Non-Leaky)...")
    df = agg_data_with_shop_features.copy()

    if 'shop_most_frequent_main_category_selected' in df.columns:
        df['is_shop_most_frequent_main_category'] = (
            df['product_main_category'] == df['shop_most_frequent_main_category_selected']
        ).astype(int)
    else:
        df['is_shop_most_frequent_main_category'] = 0 # Default if shop-level feature wasn't created

    if 'shop_most_frequent_brand_selected' in df.columns:
        df['is_shop_most_frequent_brand'] = (
            df['product_brand'] == df['shop_most_frequent_brand_selected']
        ).astype(int)
    else:
        df['is_shop_most_frequent_brand'] = 0

    # Leaky features related to selection_count_* are intentionally omitted here.
    # Examples of previously leaky features (now removed from creation):
    # df['selection_count_rank_in_shop'] = df.groupby('employee_shop')['selection_count'].rank(method='dense', ascending=False)
    # df['selection_count_share_in_shop'] = df['selection_count'] / df['total_shop_selections']
    # df['selection_count_vs_shop_avg'] = df['selection_count'] - df['shop_avg_selections_per_product_combination']

    print("Non-leaky product-relative-to-shop features created.")
    # display(df.head())
    
    final_features_df = df.copy()
    print(f"\nShape of final_features_df: {final_features_df.shape}")
    # print("Columns in final_features_df:")
    # for col in final_features_df.columns: print(f"  - {col}")
else:
    print("Skipping Product-Relative-to-Shop Features as input df is empty.")
    final_features_df = agg_data.copy() # Fallback to agg_data if previous steps failed

## 5. Final Feature Preparation for Model

In [ ]:
X = None
y = None
y_log = None
y_strata = None
label_encoders = {}

if not final_features_df.empty and 'selection_count' in final_features_df.columns:
    print("[FEATURES PREP] Preparing final X, y, y_log, y_strata...")
    y = final_features_df['selection_count']
    y_log = np.log1p(y)
    X = final_features_df.drop(columns=['selection_count']).copy()
    
    print(f"Original number of features in X before encoding: {X.shape[1]}")
    
    new_shop_categorical_features = [
        'shop_most_frequent_main_category_selected',
        'shop_most_frequent_brand_selected'
    ]
    all_categorical_cols_for_encoding = []
    if 'grouping_cols' in globals() and isinstance(grouping_cols, list):
        all_categorical_cols_for_encoding = list(set(grouping_cols + new_shop_categorical_features))
    else:
        print("Warning: grouping_cols not found or not a list. Will only use new_shop_categorical_features for determining categorical columns.")
        all_categorical_cols_for_encoding = new_shop_categorical_features

    print("\n[ENCODE] Label encoding categorical features...")
    for col in X.columns:
        if col in all_categorical_cols_for_encoding and X[col].dtype == 'object':
            X[col] = X[col].astype(str)
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col])
            label_encoders[col] = le
        elif pd.api.types.is_numeric_dtype(X[col]):
            if X[col].isnull().any():
                X[col] = X[col].fillna(X[col].median())
        elif X[col].dtype == 'object':
            X[col] = X[col].astype(str)
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col])
            label_encoders[col] = le

    X = X.fillna(0)
    for col in X.columns:
        if not pd.api.types.is_numeric_dtype(X[col]):
            print(f"Warning: Column {col} is not numeric after processing (dtype: {X[col].dtype}). Attempting conversion or dropping.")
            try:
                X[col] = pd.to_numeric(X[col])
            except ValueError:
                print(f"Could not convert {col} to numeric, dropping column.")
                X = X.drop(columns=[col])
                
    y_strata = pd.cut(y, bins=[0, 1, 2, 5, 10, np.inf], labels=[0, 1, 2, 3, 4], include_lowest=True)
    
    print(f"\nFinal X features shape: {X.shape}")
    print(f"Target shapes: Original y={y.shape}, Log y_log={y_log.shape}")
    if not y_strata.empty:
      print(f"Stratification distribution:\n{y_strata.value_counts().sort_index().to_dict()}")
    if X is not None and X.shape[1] > 0:
        print(f"Sample-to-feature ratio: {len(X) / X.shape[1]:.1f}:1")
        print("\nDEBUG: Columns in X before training:", list(X.columns))
        print(f"DEBUG: Shape of X before training: {X.shape}")
    elif X is not None:
        print("X DataFrame has 0 columns after processing.")
else:
    print("Skipping final feature preparation as final_features_df is empty or 'selection_count' is missing.")
    X = pd.DataFrame()
    y = pd.Series(dtype='float64')
    y_log = pd.Series(dtype='float64')
    y_strata = pd.Series(dtype='float64')

## 6. Model Training and Cross-Validation

In [ ]:
r2_cv_mean_log = 0.0
r2_cv_std_log = 0.0
r2_val_log = 0.0
overfitting_log = 0.0
mae_original_val = 0.0
rmse_original_val = 0.0
r2_original_val_scale = 0.0
trained_model = None

if not X.empty and not y_log.empty and not y_strata.empty:
    print("\n[MODEL TRAINING] Training XGBoost with new shop features...")
    optimal_xgb_params = {
        'n_estimators': 1000, 'max_depth': 6, 'learning_rate': 0.03,
        'subsample': 0.9, 'colsample_bytree': 0.9, 'reg_alpha': 0.3, 'reg_lambda': 0.3,
        'gamma': 0.1, 'min_child_weight': 8, 'random_state': 42, 'n_jobs': -1
    }
    model = XGBRegressor(**optimal_xgb_params)

    # Stratified Cross-Validation
    cv_stratified = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # Ensure no NaNs in strata for CV splitting
    valid_cv_indices = y_strata.dropna().index
    X_for_cv = X.loc[valid_cv_indices]
    y_log_for_cv = y_log.loc[valid_cv_indices]
    y_strata_for_cv = y_strata.loc[valid_cv_indices]

    if len(X_for_cv) > 0 and len(y_strata_for_cv) == len(X_for_cv):
        cv_scores = cross_val_score(model, X_for_cv, y_log_for_cv, 
                                    cv=cv_stratified.split(X_for_cv, y_strata_for_cv), 
                                    scoring='r2')
        r2_cv_mean_log = cv_scores.mean()
        r2_cv_std_log = cv_scores.std()
        print(f"Stratified CV R² (on y_log): {r2_cv_mean_log:.4f} ± {r2_cv_std_log:.4f}")
    else:
        print("Not enough valid data for CV after handling NaNs in strata or strata mismatch.")

    # Train/Validation Split for other metrics
    # Ensure stratify uses a clean y_strata that matches X and y_log length
    # We'll use the same valid_cv_indices for consistency if there were NaNs
    X_train, X_val, y_log_train, y_log_val, y_original_train, y_original_val = train_test_split(
        X_for_cv, y_log_for_cv, y.loc[valid_cv_indices], 
        test_size=0.2, random_state=42, stratify=y_strata_for_cv
    )
    
    trained_model = XGBRegressor(**optimal_xgb_params)
    trained_model.fit(X_train, y_log_train)
    
    y_log_pred_val = trained_model.predict(X_val)
    r2_val_log = r2_score(y_log_val, y_log_pred_val)
    print(f"Validation R² (on y_log): {r2_val_log:.4f}")
    
    overfitting_log = r2_val_log - r2_cv_mean_log
    print(f"Overfitting (Validation R² - CV R²): {overfitting_log:+.4f}")

    # Metrics on Original Scale
    y_pred_val_original_scale = np.expm1(y_log_pred_val)
    y_pred_val_original_scale = np.maximum(0, y_pred_val_original_scale)
    
    mae_original_val = mean_absolute_error(y_original_val, y_pred_val_original_scale)
    mse_original_val = mean_squared_error(y_original_val, y_pred_val_original_scale)
    rmse_original_val = np.sqrt(mse_original_val)
    r2_original_val_scale = r2_score(y_original_val, y_pred_val_original_scale)

    print(f"\nMetrics on Original Scale (Validation Set):")
    print(f"  MAE: {mae_original_val:.4f}")
    print(f"  RMSE: {rmse_original_val:.4f}")
    print(f"  R² (original scale): {r2_original_val_scale:.4f}")
    
    baseline_r2 = 0.2947 # From original breakthrough
    improvement = r2_cv_mean_log - baseline_r2
    print(f"\nImprovement in CV R² (log target) over baseline ({baseline_r2}): {improvement:+.4f}")
else:
    print("Skipping Model Training as X, y_log or y_strata is empty/invalid.")

## 7. Feature Importance Analysis

In [ ]:
if trained_model is not None and not X.empty:
    print("\n[ANALYSIS] Feature Importance Analysis")
    feature_importances = trained_model.feature_importances_
    feature_names = X.columns # Use columns from the final X used for training
    
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': feature_importances
    }).sort_values('importance', ascending=False)
    
    print("\nFeature Importance Ranking (New Model with Shop Features):")
    for i, row in importance_df.iterrows():
        print(f"  {i+1:2d}. {row['feature']:<45} {row['importance']:.4f}")
    
    # Plot feature importance
    plt.figure(figsize=(12, max(8, len(importance_df) * 0.3)))
    sns.barplot(x='importance', y='feature', data=importance_df, palette='viridis')
    plt.title('XGBoost Feature Importance - Model with Shop Features')
    plt.tight_layout()
    plt.show()
else:
    print("Skipping feature importance as model was not trained or X is empty.")

## 8. Model Persistence (Optional)

In [ ]:
save_model_threshold = 0.30 # Save if CV R2 (log) is better than this
if trained_model is not None and r2_cv_mean_log > save_model_threshold:
    print(f"\n[SAVE] New model performance ({r2_cv_mean_log:.4f}) exceeds threshold ({save_model_threshold:.4f}). Saving model...")
    
    model_dir = '../models/final_shop_features_model/' # New directory for this version
    os.makedirs(model_dir, exist_ok=True)
    
    final_model_path = os.path.join(model_dir, 'final_xgb_shop_features_model.pkl')
    joblib.dump(trained_model, final_model_path)
    print(f"[OK] Final model saved to: {final_model_path}")
    
    final_encoders_path = os.path.join(model_dir, 'final_label_encoders.pkl')
    with open(final_encoders_path, 'wb') as f:
        pickle.dump(label_encoders, f) 
    print(f"[OK] Final label encoders saved to: {final_encoders_path}")
    
    final_metadata = {
        'model_type': 'XGBoost Regressor',
        'description': 'Model trained with refined shop assortment features (non-leaky)',
        'source_data_aggregation': 'present.selection.historic.csv aggregated by 11 features',
        'target_transformation': 'log1p(selection_count)',
        'cv_methodology': 'StratifiedKFold (5 splits) by selection_count bins',
        'performance_log_target': {
            'stratified_cv_r2_mean': r2_cv_mean_log,
            'stratified_cv_r2_std': r2_cv_std_log,
            'validation_r2': r2_val_log,
            'overfitting': overfitting_log
        },
        'performance_original_scale_validation': {
            'mae': mae_original_val,
            'rmse': rmse_original_val,
            'r2': r2_original_val_scale
        },
        'features': list(X.columns) if not X.empty else [],
        'training_data_size': len(X_train) if 'X_train' in locals() and not X_train.empty else (len(X_for_cv) if 'X_for_cv' in locals() else 0),
        'feature_importance': importance_df.set_index('feature')['importance'].to_dict() if 'importance_df' in locals() and not importance_df.empty else {}
    }
    final_metadata_path = os.path.join(model_dir, 'final_model_metadata.pkl')
    with open(final_metadata_path, 'wb') as f:
        pickle.dump(final_metadata, f)
    print(f"[OK] Final model metadata saved to: {final_metadata_path}")
else:
    if trained_model is not None:
        print(f"\n[NO SAVE] New model performance ({r2_cv_mean_log:.4f}) does not exceed threshold ({save_model_threshold:.4f}). Model not saved.")
    else:
        print("\n[NO SAVE] Model not trained or performance metrics unavailable. Model not saved.")

## 9. Summary of Final Model with Shop Features

In [ ]:
print("\n" + "="*80)
print("FINAL MODEL RESULTS (with Shop Assortment Features - Non-Leaky)")
print("="*80 + "\n")
if trained_model is not None:
    print(f"Stratified CV R² (log target): {r2_cv_mean_log:.4f} ± {r2_cv_std_log:.4f}")
    print(f"Validation R² (log target): {r2_val_log:.4f}")
    print(f"Overfitting (log target): {overfitting_log:+.4f}")
    print("\nMetrics on Original Scale (Validation Set):")
    print(f"  MAE: {mae_original_val:.4f}")
    print(f"  RMSE: {rmse_original_val:.4f}")
    print(f"  R² (original scale): {r2_original_val_scale:.4f}")
    
    baseline_r2 = 0.2947
    improvement = r2_cv_mean_log - baseline_r2
    print(f"\nImprovement in CV R² (log target) over baseline ({baseline_r2}): {improvement:+.4f}")
    
    if improvement > 0.01:
        print("\n[CONCLUSION] The new shop assortment features provided a notable improvement.")
    elif improvement > 0:
        print("\n[CONCLUSION] The new shop assortment features provided a slight improvement.")
    else:
        print("\n[CONCLUSION] The new shop assortment features did not significantly improve or worsened performance compared to baseline.")
else:
    print("Model training was not completed successfully. Cannot display final results.")